# Three phase transmission line - one cable per phase
### Author :  Carlos K.P.S.
### 2019.1 UFRJ - Brazil - electrical engeneering student

In [176]:
import numpy as np                                                   # for numerical calculus
import sympy                                                         # symbolic library
from sympy.abc import x, y                                           # in all code x and y are treated as symbolic variables
from sympy.utilities.lambdify import lambdify, implemented_function  # for convert from symbolic to numerical
from ipywidgets import interactive                                   # to make the graph's interativity
from matplotlib import cm
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)

In [177]:
u = 1.0              # 1pu       
rf = 10e-3           # the conductor radius
xc = [-7,0,7]        # define the x distance of all conductors
yc = [10,10,10]      # define the y distance between eacho conductor to the ground

In [185]:
# define the volts function. The entrance is a angle and its return a lis conteined a value of each fase in terms of the 
# pahse angle
def volts(theta):
    angle = np.radians(theta)                                                         # transform the degree angle into radian
    volts = [u*np.round(np.cos(angle),8),u*np.cos(angle-2.*np.pi/3),u*np.round(np.cos(angle-4.*np.pi/3),8)]
    return volts

In [186]:
# the function that calculates the Maxwell potential matrix
def pot_matrix(x,y,rf):
    matrix = []
    for i in range(0,len(xc)):
        aux_list = []
        for j in range(0,len(yc)):
            if i!=j:
                calculo = 0.5*np.log(((y[i]+y[j])**2+(x[i]-x[j])**2)/((y[i]-y[j])**2+(x[i]-x[j])**2))
                aux_list.append(calculo)
            else:
                calculo = np.log(2*y[i]/rf)
                aux_list.append(calculo)
        matrix.append(aux_list)
    return matrix

In [187]:
# define a list of the equivalent charges 
def tot_charge(theta):
    return np.linalg.inv(pot_matrix(xc,yc,rf))@volts(theta) # @ is the product of tow matrix

In [188]:
# define the potential function
def potencial(theta):
    pot_aux = 0   # create a auxiliary variable for the iteration below
    char = tot_charge(theta)
    for i in np.arange(0,len(char)):  
        pot_aux += (char[i])*1./(((x-xc[i])**2+(y-yc[i])**2)**0.5) - (char[i])/(((x-xc[i])**2+(y+yc[i])**2)**0.5) 
    return pot_aux

In [189]:
# define the components of eletric fields
def field(theta):
    pot = potencial(theta)
    ex = -sympy.diff(pot,x)
    ey = -sympy.diff(pot,y)
    return [ex,ey]

In [190]:
# create meshgrid xx and yy
xx,yy=np.meshgrid(np.linspace(-20,20,80),np.linspace(-20,20,150))

In [192]:
# define the plot
def plotting(theta=0):
    
    pot_sym =potencial(theta)                     # gives a symbolic potential function 
    pot_num = lambdify((x,y),pot_sym,'numpy')     # convert into a numerical fucntion
    
    eletric_field = field(theta)                  # calling the field function tha gives the symbolical electric field components
    ex = lambdify((x,y),eletric_field[0],'numpy') # electric field in x direction
    ey = lambdify((x,y),eletric_field[1],'numpy') # electric field in y direction
    
   
    
    fig, ax = plt.subplots(figsize=(13,13))       # choosing the plot size
    #plt.contour(xx,yy,pot_num(xx,yy),950,colors='darkblue',linestyles='-')
    cont1 = plt.contour(xx,yy,pot_num(xx,yy),750,cmap='gist_gray') 
    plt.colorbar(cont1)
    #plt.contourf(xx,yy,pot_num(xx,yy),100)
    ax.streamplot(xx,yy,ex(xx,yy),ey(xx,yy),density=2,arrowsize=0.7,color='r')
    
    #cp = plt.contourf(xx, yy, pot_num(xx,yy),300,cmap='nipy_spectral')
    #plt.colorbar(cp)
    
    plt.xlim(-20,20)                   
    plt.ylim(-20,20)
    plt.show()
    
    
interative_plot = interactive(plotting,theta=(0,360,30))
interative_plot

interactive(children=(IntSlider(value=0, description='theta', max=360, step=30), Output()), _dom_classes=('wid…

In [24]:
tot_charge(90)

array([-0.01,  0.13, -0.13])